## The MiniVGGNet architecture


In [1]:
# dlframework.nn.conv.minivggnet.py

from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Activation

from keras.layers.normalization import BatchNormalization
from keras import backend as K


In [ ]:
class MiniVGGNet:
    @staticmethod
    def build( width, height, depth, classes):
        
        #initialize the model
        model = Sequential()
        inputShape = (height, width, depth)
        channelDim = -1
        
        # if we are using "channels first", update the input shape
        if K.image_data_format() == "channel_first":
            inputShape = (depth, height, width)
            channelDim = 1
            
        # first CONV => RELU => CONV => RELU => POOL layer set
        model.add(Conv2D(32, (3,3), padding="same",
                        input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis = channelDim))
        model.add(Conv2D(32, (3,3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis = channelDim))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.25))
        
        # second CONV => RELU => CONV => RELU => POOL layer set
        model.add(Conv2D(64, (3,3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis = channelDim))
        model.add(Conv2D(64, (3,3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis = channelDim))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        model.add(Dropout(0.25))
        
        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
        return model
                  

## MiniVGGNet on CIFAR-10 (with BN)

In [3]:
# miniVGGNet_CIFAR10.py
from sklearn.preprocessing import LabelBinarizer #One-Hot encoder
from sklearn.model_selection import train_test_split
from dlframework.nn.conv import MiniVGGNet
from keras.optimizers import SGD
from keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np
import argparse


# construct the argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-o", "--output", required=True,
#                 help="path to the output loss/accuracy plot")
# args = vars(ap.parse_args())



In [5]:
# load the training and testing data, then scale it into the
# range [0, 1]
print("[INFO] loading CIFAR-10 data...")
((X_train, y_train), (X_test, y_test)) = cifar10.load_data()
X_train = X_train.astype("float")/255.0
X_test = X_test.astype("float")/255.0

#one-hot encode the labels
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)
labels= ["airplane", "automobile", "bird", "cat", "deer",
        "dog", "frog", "horse", "ship", "truck"]


[INFO] loading CIFAR-10 data...


In [9]:
print("[INFO] compiling model...")
model = MiniVGGNet.build(width=32, height=32, depth=3, classes=10)
opt = SGD(lr=0.01, decay=0.01/40, momentum=0.9,  nesterov=True)  #Nestrov's accelerated GD
model.compile(loss="categorical_crossentropy", optimizer=opt,
                metrics =["accuracy"])

H = model.fit(X_train, y_train,  validation_data=(X_test, y_test),
            batch_size=64, epochs=40, verbose=1)

[INFO] compiling model...
Train on 50000 samples, validate on 10000 samples
Epoch 1/40
50000/50000 [==============================] - 315s 6ms/step - loss: 1.5877 - accuracy: 0.4701 - val_loss: 1.2559 - val_accuracy: 0.5639
Epoch 2/40
50000/50000 [==============================] - 191s 4ms/step - loss: 1.0994 - accuracy: 0.6183 - val_loss: 0.9699 - val_accuracy: 0.6550
Epoch 3/40
50000/50000 [==============================] - 192s 4ms/step - loss: 0.9362 - accuracy: 0.6737 - val_loss: 0.7972 - val_accuracy: 0.7241
Epoch 4/40
50000/50000 [==============================] - 190s 4ms/step - loss: 0.8376 - accuracy: 0.7070 - val_loss: 0.7500 - val_accuracy: 0.7372
Epoch 5/40
50000/50000 [==============================] - 186s 4ms/step - loss: 0.7675 - accuracy: 0.7297 - val_loss: 0.7359 - val_accuracy: 0.7407
Epoch 6/40
26880/50000 [===============>..............] - ETA: 1:24 - loss: 0.7134 - accuracy: 0.7492

KeyboardInterrupt: 